In [1]:
import pandas
import numpy
import math

In [2]:
#Parse covariance matrix data
cov = pandas.read_csv('./dataset/DS1_Cov.txt', header=None)
cov.drop([20], axis=1, inplace=True)
cov = cov.values

#Parse the mean vectors
m0 = numpy.genfromtxt('./dataset/DS1_m_0.txt', delimiter=',')[:-1]
m1 = numpy.genfromtxt('./dataset/DS1_m_1.txt', delimiter=',')[:-1]

In [3]:
ds1 = []
ds0 = []
for i in range(2000):
    ds1.append(numpy.random.multivariate_normal(m1, cov))
    ds0.append(numpy.random.multivariate_normal(m0, cov))
ds1 = pandas.DataFrame(ds1)
ds0 = pandas.DataFrame(ds0)
ds0[20] = 0
ds1[20] = 1

msk = numpy.random.rand(len(ds1)) <= 0.7
train1 = ds1[msk]
test1 = ds1[~msk]

msk = numpy.random.rand(len(ds0)) <= 0.7
train0 = ds0[msk]
test0 = ds0[~msk]

train = pandas.concat([train0, train1], ignore_index=True)
test = pandas.concat([test0, test1], ignore_index=True)

train1.to_csv('DS1_train1.csv')
test1.to_csv('DS1_test1.csv')
train0.to_csv('DS1_train0.csv')
test0.to_csv('DS1_test0.csv')

test.to_csv('DS1_test.csv')
train.to_csv('DS1_train.csv')

In [4]:
len0 = len(train0)
len1 = len(train1)


# drop the output column
train0 = train[train[20] == 0]
train0 = train0[train0.columns.difference([20])]

train1 = train[train[20] == 1]
train1 = train1[train1.columns.difference([20])]

test_output = test[20]
test.drop([20], axis=1, inplace=True)

train_output = train[20]
train.drop([20], axis=1, inplace=True)

In [5]:
# estimated max probability
prob_0 = float(len0) / float(len0 + len1)
prob_1 = 1.0 - prob_0


mean_0 = numpy.array(train0.mean())
mean_1 = numpy.array(train1.mean())

diff_0 = numpy.array(train0 - mean_0)
diff_1 = numpy.array(train1 - mean_1)

coeff_0 = numpy.matmul(diff_0.T, diff_0)
coeff_1 = numpy.matmul(diff_1.T, diff_1)

coeff = (coeff_0 + coeff_1) / float(len0 + len1)

covm_0 = numpy.matmul(numpy.linalg.pinv(coeff), mean_0)
mcovm_0 = numpy.matmul(mean_0.T, covm_0)

covm_1 = numpy.matmul(numpy.linalg.pinv(coeff), mean_1)
mcovm_1 = numpy.matmul(mean_1.T, covm_1)

w0 = math.log(prob_0) - math.log(prob_1) - 1 / 2 * (mcovm_0 - mcovm_1)
w1 = numpy.matmul(numpy.linalg.pinv(coeff), mean_0 - mean_1)


print("w0: ", w0)
print("w1: " + str([i for i in w1]) + "\n")

pred = numpy.matmul(test, w1) + w0

# replace dec_bound to 0 and 1
pred[pred > 0] = 0
pred[pred < 0] = 1

error_matrix = pred - test_output

elem, cnt = numpy.unique(error_matrix, return_counts=True)

print(numpy.asarray([elem, cnt]))

'''
1 => false positive
0 => correct prediction
-1 => false negative
'''

fp = cnt[2]
fn = cnt[0]


tp = numpy.sum(pred.dot(test_output))
print("True positive: " + str(tp))

pred = pred - 1
test_output_temp = test_output - 1
tn = numpy.sum(pred.dot(test_output_temp))
print("True negative: " + str(tn))

precision = tp / (tp + fp)
recall = tp / (tp + fn)
accuracy = (tn + tp) / (tn + tp + fn + fp)

F = 2 * precision * recall / (precision + recall)

print("\nF measure: " + str(F))
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))

w0:  27.8476877165
w1: [14.707729156895715, -8.8305406001678168, -5.5221547556501562, -2.6214071915318229, -10.02977954152756, -4.1847592094945387, 16.79210358829441, -25.039403079155537, -29.697412075196127, 9.5566884645144761, -13.382001513262969, -12.230248062200609, 15.742262171088065, 13.102390147524336, -5.9258220337026728, 13.480181549007739, 29.549738076362232, -6.9738799729481098, 0.013707430910422147, -5.3335266414797884]

[[ -1.00000000e+00   0.00000000e+00   1.00000000e+00]
 [  2.60000000e+01   1.16300000e+03   2.70000000e+01]]
True positive: 579.0
True negative: 584.0

F measure: 0.956234516928
Accuracy: 0.956414473684
Precision: 0.955445544554
Recall: 0.957024793388


In [7]:
res_step = []
accuracy_k = []


for k in range(1, 50):
    # metrics = [tp, tn, fp, fn]
    metrics = [0, 0, 0, 0]
    for i in range(len(test)):
        dist = abs(train.sub(numpy.array(numpy.array(test.loc[[i], :])[0])))
        dist = numpy.array(numpy.power(dist, 2).sum(axis=1))

        # select the k training examples closest to the test example
        ind = numpy.argpartition(dist, k)[:k]

        # predict value of test output using the average of the k training outputs
        dec = 1 if (numpy.array([train_output[i] for i in ind]).mean() > 0.5) else 0

        #update metrics
        if dec == test_output[i] == 1:
            metrics[0] += 1
        elif dec == test_output[i] == 0:
            metrics[1] += 1
        elif dec != test_output[i] == 1:
            metrics[2] += 1
        else:
            metrics[3] += 1

    res_step.append(metrics)
    accuracy_k.append((metrics[0] + metrics[1]) / numpy.sum(metrics))


print("Accuracy per k: ", accuracy_k)

print("\nBest k = " + str(accuracy_step.index(max(accuracy_k)) + 1))

metrics = res_step[accuracy_k.index(max(accuracy_k))]
tp = metrics[0]
tn = metrics[1]
fp = metrics[2]
fn = metrics[3]

precision = tp / (tp + fp)
recall = tp / (tp + fn)
accuracy = (tn + tp) / (tn + tp + fn + fp)

F = 2 * precision * recall / (precision + recall)

print("\nAccuracy = " + str(max(accuracy_step)))
print("Precision = " + str(precision))
print("Recall = " + str(recall))
print("F - measure = " + str(F))




Accuracy per k:  [0.51315789473684215, 0.51233552631578949, 0.54523026315789469, 0.52302631578947367, 0.52138157894736847, 0.52467105263157898, 0.52878289473684215, 0.53782894736842102, 0.53700657894736847, 0.53125, 0.52549342105263153, 0.51644736842105265, 0.52796052631578949, 0.53289473684210531, 0.52713815789473684, 0.52467105263157898, 0.54440789473684215, 0.53371710526315785, 0.53700657894736847, 0.52384868421052633, 0.53207236842105265, 0.54194078947368418, 0.54605263157894735, 0.53947368421052633, 0.55263157894736847, 0.54934210526315785, 0.54769736842105265, 0.55098684210526316, 0.53947368421052633, 0.54605263157894735, 0.55674342105263153, 0.55345394736842102, 0.55427631578947367, 0.56003289473684215, 0.55592105263157898, 0.56003289473684215, 0.55345394736842102, 0.55838815789473684, 0.5625, 0.5625, 0.5625, 0.5625, 0.56085526315789469, 0.57072368421052633, 0.56085526315789469, 0.57072368421052633, 0.57319078947368418, 0.56825657894736847, 0.57154605263157898]

Best k = 47

Acc